In [6]:
from itertools import islice
from pyexpat import features

import pandas as pd
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern, hog
from skimage.measure import shannon_entropy

In [7]:
train_df = pd.read_csv("./dataset/splits/train.csv", index_col='index')
test_df = pd.read_csv("./dataset/splits/test.csv", index_col='index')
val_df = pd.read_csv("./dataset/splits/validation.csv", index_col='index')
train_df.head()

,clip_name,clip_path,label,encoded_label
index,,,,
0,v_Diving_g03_c01.avi,./dataset/Diving/v_Diving_g03_c01.avi,Diving,0
1,v_Diving_g19_c03.avi,./dataset/Diving/v_Diving_g19_c03.avi,Diving,0
2,v_Diving_g03_c04.avi,./dataset/Diving/v_Diving_g03_c04.avi,Diving,0
3,v_Diving_g05_c04.avi,./dataset/Diving/v_Diving_g05_c04.avi,Diving,0
4,v_Diving_g15_c03.avi,./dataset/Diving/v_Diving_g15_c03.avi,Diving,0


In [8]:
CONFIG = {
    'frame_skip': 5,  # Process every 5th frame to speed up
    'resize_dim': (224, 224),
    'n_jobs': -1,  # Use all CPU cores for feature extraction

    'lbp_radius': 3,
    'lbp_points': 8,

    'gabor': {
        'ksize': 5,
        'sigma': 3,
        'theta': 1*np.pi/4,
        'lamda': 1*np.pi /4,
        'gamma':0.4,
        'phi': 0
    },

    'contour': {
        'count' : 3,
    },

    'lucas_kanade': {
        'max_corners': 10,
        'quality_level': .3,
        'min_distance': 7,
        'block_size': 7
    },
}

In [9]:
class VideoExtractorFeature:
    def __init__(self, skip_frame:int =5):
        self.skip_frame = skip_frame
        self.scalar = MinMaxScaler()
        self.gabor_kernel = cv.getGaborKernel(
            (CONFIG['gabor']['ksize'], CONFIG['gabor']['ksize']),
            CONFIG['gabor']['sigma'],
            CONFIG['gabor']['theta'],
            CONFIG['gabor']['lamda'],
            CONFIG['gabor']['gamma'],
            CONFIG['gabor']['phi'],
        )

    def _get_frame_color_features(self, frame):
        hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        features = {}
        for i, column_name in enumerate(['h', 's', 'v']):
            channel = hsv_frame[:, :, i]

            mean = np.mean(channel)
            std = np.std(channel)

            if std < 1e-6:
                skewness = 0
            else:
                skewness = skew(channel.flatten())
                if np.isnan(skewness):
                    skewness = 0

            features[f'moments_mean_{column_name}'] = mean
            features[f'moments_std_{column_name}'] = std
            features[f'moments_skew_{column_name}'] = skewness

        avg_rgb  = np.mean(rgb_frame, axis=(0, 1))
        features['avg_color_r'] = avg_rgb[0]
        features['avg_color_g'] = avg_rgb[1]
        features['avg_color_b'] = avg_rgb[2]
        return features

    def _get_frame_glcm_features(self, grey_frame):
        features = {}
        for distance in [1, 3, 5]:
            for angle, orientation in zip([0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], ["0", "45", "90", "135"]):
                glcm = graycomatrix(grey_frame, distances=[distance], angles=[angle], levels=256, symmetric=True, normed=True)
                features[f'glcm_contrast_{distance}_{orientation}'] = graycoprops(glcm, 'contrast')[0, 0]
                features[f'glcm_dissimilarity_{distance}_{orientation}'] = graycoprops(glcm, 'dissimilarity')[0, 0]
                features[f'glcm_homogeneity_{distance}_{orientation}'] = graycoprops(glcm, 'homogeneity')[0, 0]
                features[f'glcm_correlation_{distance}_{orientation}'] = graycoprops(glcm, 'correlation')[0, 0]
                features[f'glcm_energy_{distance}_{orientation}'] = graycoprops(glcm, 'energy')[0, 0]
        features['glcm_entropy'] = shannon_entropy(grey_frame)
        return features

    def _lbp_features(self, grey_frame):
        lbp = local_binary_pattern(grey_frame, CONFIG['lbp_points'], CONFIG['lbp_radius'], method='uniform')
        hist, _ = np.histogram(lbp.ravel(),
                               bins=np.arange(0, CONFIG['lbp_points'] + 3),
                               range=(0, CONFIG['lbp_points'] + 2))
        hist = hist.astype('float')
        hist /= (hist.sum() + 1e-7)

        features = {}
        for i in range(len(hist)):
            features[f'lbp_{i}'] = hist[i]
        return features

    def _get_gabor_features(self, grey_frame):
        gabor_features = cv.filter2D(grey_frame, cv.CV_8UC3, self.gabor_kernel)

        hist, _ = np.histogram(gabor_features.ravel(), bins=10,)
        hist = hist.astype('float')
        hist /= (hist.sum() + 1e-7)

        features = {}
        for i in range(len(hist)):
            features[f'gabor_{i}'] = hist[i]
        return features

    def _get_canny_features(self, grey_frame):
        sigma = 0.3
        median = np.median(grey_frame)
        lower = int(max(0, (1.0 - sigma) * median))
        upper = int(min(255, (1.0 + sigma) * median))
        edges = cv.Canny(grey_frame, lower, upper, edges=grey_frame)

        hist, _ = np.histogram(edges.ravel(), bins=10,)
        hist = hist.astype('float')
        hist /= (hist.sum() + 1e-7)

        features = {}
        for i in range(len(hist)):
            features[f'canny_{i}'] = hist[i]
        return features

    def _get_contour_features(self, grey_frame):
        sigma = 0.2
        median = np.median(grey_frame)
        lower = int(max(0, (1.0 - sigma) * median))
        upper = int(min(255, (1.0 + sigma) * median))
        ret, img_th = cv.threshold(grey_frame, lower, upper, cv.THRESH_BINARY_INV)
        contours, hierarchy = cv.findContours(img_th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        features = {}
        for i in range(CONFIG['contour']['count']):
            features[f'contour_{i}_area'] = -1
            features[f'contour_{i}_perimeter'] = -1

        for i, contour in enumerate(islice(contours, min(CONFIG['contour']['count'], len(contours)))):
            m = cv.moments(contour)
            features[f'contour_{i}_area'] = m['m00']
            features[f'contour_{i}_perimeter'] = m['m01'] / (m['m00'] + 1e-7)
            # 'contour_aspect_ratio': m['m10'] / m['m00'],
            # 'contour_eccentricity': cv.contourArea(contours[0]) / cv.arcLength(contours[0], True),
            # 'contour_extent': cv.contourArea(contours[0]) / (grey_frame.shape[0] * grey_frame.shape[1]),
            # 'contour_solidity': cv.contourArea(contours[0]) / cv.contourArea(cv.convexHull(contours[0])),
        return features

    def _get_hog_features(self, grey_frame):
        features = {}
        feature = hog(grey_frame, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1))
        for i in range(len(feature)):
            features[f'hog_{i}'] = feature[i]
        return features

    def _get_lucas_kanade_features(self, prev_frame, curr_frame):
        p0 = cv.goodFeaturesToTrack(prev_frame,
                               mask=None, maxCorners=CONFIG['lucas_kanade']['max_corners'], qualityLevel=CONFIG['lucas_kanade']['quality_level'], minDistance=CONFIG['lucas_kanade']['min_distance'], blockSize=CONFIG['lucas_kanade']['block_size'])
        if p0 is None:
            feature = np.zeros(CONFIG['lucas_kanade']['max_corners'] * 2)
        else:
            lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
            p1, st, err = cv.calcOpticalFlowPyrLK(prev_frame, curr_frame, p0, None, **lk_params)
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            displacements = good_new - good_old
            if len(displacements) < CONFIG['lucas_kanade']['max_corners']:
                padding = np.zeros((CONFIG['lucas_kanade']['max_corners'] - len(displacements), 2))
                displacements = np.concatenate((displacements, padding), axis=0)
            else:
                displacements = displacements[:CONFIG['lucas_kanade']['max_corners']]

            feature = displacements.flatten()

        features = {}
        for i in range(len(feature)):
            features[f'lk_{i}'] = feature[i]
        return features

    def _get_motion_features(self, prev_frame, curr_frame):
        # flow = self.hs_flow.calc(prev_frame, curr_frame, None)
        flow = cv.calcOpticalFlowFarneback(
            prev_frame, curr_frame, None,
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0,
        )

        u = flow[..., 0]
        v = flow[..., 1]

        mag, ang = cv.cartToPolar(u, v, angleInDegrees=True)
        mag_hist, _ = np.histogram(mag, bins=10, range=(0, 10))
        ang_hist, _ = np.histogram(ang, bins=8, range=(0, 360), weights=mag)
        avg_intensity = np.mean(mag)
        dominant_direction = np.argmax(ang_hist) * 45  # Back to degrees (8 bins = 45 deg/bin)

        features = {}
        for i in range(len(mag_hist)):
            features[f'motion_magnitude_{i}'] = mag_hist[i]
        for i in range(len(ang_hist)):
            features[f'motion_direction_{i}'] = ang_hist[i]
        features['motion_avg_intensity'] = avg_intensity
        features['motion_dominant_direction'] = dominant_direction
        return features

    def _process_video(self, row: dict):
        video_id = row['index']
        video_path = row['clip_path']
        cap = cv.VideoCapture(video_path)
        features = []
        frame_cnt = 0
        prev_grey_frame = None

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_cnt % self.skip_frame == 0:
                frame = cv.resize(frame, CONFIG['resize_dim'], interpolation=cv.INTER_AREA)
                grey_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                if prev_grey_frame is None:
                    prev_grey_frame = grey_frame
                feat = {'index': video_id}
                feat.update(self._get_frame_color_features(frame))
                feat.update(self._get_frame_glcm_features(grey_frame))
                feat.update(self._lbp_features(grey_frame))
                feat.update(self._get_gabor_features(grey_frame))
                feat.update(self._get_canny_features(grey_frame))
                feat.update(self._get_contour_features(grey_frame))
                feat.update(self._get_hog_features(grey_frame))
                feat.update(self._get_lucas_kanade_features(prev_grey_frame, grey_frame))
                feat.update(self._get_motion_features(prev_grey_frame, grey_frame))
                features.append(feat)
                prev_grey_frame = grey_frame
            frame_cnt += 1

        cap.release()
        return features

    def _normalize_features(self, feature_df: pd.DataFrame, is_test: bool = False):
        feature_names = [col for col in feature_df.columns if col not in ['index', 'encoded_label']]
        feature_df[feature_names] = feature_df[feature_names].astype(float)
        if is_test:
            feature_df[feature_names] = self.scalar.transform(feature_df[feature_names])
        else:
            feature_df[feature_names] = self.scalar.fit_transform(feature_df[feature_names])
        return feature_df

    def _process_dataset(self, df: pd.DataFrame, is_test=False):
        rows = df.reset_index().to_dict('records')
        nested_results = Parallel(n_jobs=-1)(delayed(self._process_video)(row) for row in rows)

        flatten_results = [item for sub_list in nested_results for item in sub_list]
        feature_df = pd.DataFrame(flatten_results)
        return self._normalize_features(feature_df, is_test)

    def process_train_df(self, df: pd.DataFrame):
        process_df = self._process_dataset(df)
        merged_df = pd.merge(train_df, process_df, on='index', how='inner')
        output = merged_df['encoded_label']
        merged_df = merged_df.drop(columns=['index', 'clip_path', 'clip_name', 'label', 'encoded_label'])
        return merged_df, output

    def process_test_df(self, df: pd.DataFrame):
        process_df = self._process_dataset(df, True)
        merged_df = pd.merge(train_df, process_df, on='index', how='inner')
        merged_df = merged_df.drop(columns=['clip_path', 'clip_name', 'label'])
        return merged_df


In [10]:
extractor = VideoExtractorFeature()

In [11]:
X, Y = extractor.process_train_df(train_df)

In [12]:
M = extractor.process_test_df(val_df)

In [13]:
X.head()

,moments_mean_h,moments_std_h,moments_skew_h,moments_mean_s,moments_std_s,moments_skew_s,moments_mean_v,moments_std_v,moments_skew_v,avg_color_r,...,motion_direction_0,motion_direction_1,motion_direction_2,motion_direction_3,motion_direction_4,motion_direction_5,motion_direction_6,motion_direction_7,motion_avg_intensity,motion_dominant_direction
0,0.437377,0.703766,0.469759,0.184177,0.320461,0.443086,0.672033,0.708757,0.265971,0.621076,...,0.000004,0.000008,0.000001,0.000006,0.000008,0.000002,0.000001,0.000005,0.000029,0.142857
1,0.419381,0.691181,0.472115,0.173806,0.312256,0.452479,0.671253,0.707400,0.266933,0.624704,...,0.019859,0.041062,0.038140,0.013508,0.007470,0.001588,0.000636,0.003928,0.105426,0.285714
2,0.445359,0.637138,0.472924,0.174393,0.315061,0.467496,0.670058,0.709306,0.268808,0.624340,...,0.008720,0.039702,0.023652,0.017200,0.025327,0.011158,0.006968,0.015411,0.120252,0.142857
3,0.452045,0.706548,0.463693,0.168384,0.309778,0.468080,0.672741,0.704658,0.267192,0.630040,...,0.012389,0.013392,0.032040,0.025171,0.005335,0.004400,0.002500,0.001263,0.080770,0.285714
4,0.458268,0.636646,0.467077,0.161103,0.303536,0.483310,0.673845,0.710806,0.268158,0.634046,...,0.008576,0.011658,0.069417,0.026908,0.011401,0.004632,0.006077,0.004908,0.123172,0.285714


In [14]:
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: encoded_label, dtype: int64

In [15]:
feature_names = [col for col in M.columns if col not in ['index', 'encoded_label']]

In [16]:
import optuna.visualization as vis
import cupy as cp
import optuna
from cuml.svm import SVC
import cudf

def validation_pipeline_score(model, merged_df):
    result = 0
    for video_id, group in merged_df.groupby('index'):
        df = group[feature_names]
        output = model.predict(df)
        final_ans = output.mode().iloc[0]
        expected = group['encoded_label'].values[0]
        # print(f"final_ans: {final_ans}, expected: {expected}, type of expected: {type(expected)}, type of final_ans: {type(final_ans)}, video_id: {video_id}")
        if final_ans == expected:
            result = result + 1
    return result


def objective_function(trial):
    params = {
        'C': trial.suggest_float('C', 1e-3, 1e3, log=True),
        'gamma': trial.suggest_float('gamma', 1e-4, 1e1, log=True),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    }
    model = SVC(**params)
    model.fit(X, Y)
    return validation_pipeline_score(model, M)


study = optuna.create_study(direction="maximize")
study.optimize(objective_function, n_trials=50, show_progress_bar=True, n_jobs=1)


[I 2026-02-05 22:00:11,069] A new study created in memory with name: no-name-1bf16b8c-45c2-4471-a0ca-3b1a58d03eeb


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-02-05 22:00:12,872] Trial 0 finished with value: 10.0 and parameters: {'C': 0.06346024936329484, 'gamma': 0.001690509802142077, 'kernel': 'linear'}. Best is trial 0 with value: 10.0.
[I 2026-02-05 22:00:13,944] Trial 1 finished with value: 10.0 and parameters: {'C': 27.15264762777522, 'gamma': 0.04226452441696524, 'kernel': 'poly'}. Best is trial 0 with value: 10.0.
[I 2026-02-05 22:00:14,998] Trial 2 finished with value: 10.0 and parameters: {'C': 0.019709537705866455, 'gamma': 0.022017713484422007, 'kernel': 'poly'}. Best is trial 0 with value: 10.0.
[I 2026-02-05 22:00:20,999] Trial 3 finished with value: 16.0 and parameters: {'C': 63.58700011239925, 'gamma': 0.20036671217962895, 'kernel': 'rbf'}. Best is trial 3 with value: 16.0.
[I 2026-02-05 22:00:22,096] Trial 4 finished with value: 10.0 and parameters: {'C': 1.410682001491023, 'gamma': 0.015136368202312048, 'kernel': 'linear'}. Best is trial 3 with value: 16.0.
[I 2026-02-05 22:00:23,103] Trial 5 finished with value: 10

In [17]:
vis.plot_param_importances(study)

In [18]:
vis.plot_optimization_history(study)